In [1]:
import warnings
import numpy as np
import h5py

import os

In [2]:
paths = []
for dirpath, dirnames, filenames in os.walk("/home/lorenzo/phd/LQG/ForwardDelta/sims"):
    if "/outputs" in dirpath:
        paths.append(dirpath)

print(paths)

['/home/lorenzo/phd/LQG/ForwardID/sims/m5.0_dx0.005_xMax70.0_tf400.0_r030.0_a080.0/outputs', '/home/lorenzo/phd/LQG/ForwardID/sims/m5.0_dx0.005_xMax70.0_tf200.0_r02.14_a05.7066/outputs', '/home/lorenzo/phd/LQG/ForwardID/sims/m5.0_dx0.01_xMax70.0_tf200.0_r02.14_a05.7066/outputs']


In [3]:
for path in paths:

    with h5py.File(path + '/output.h5', 'w') as f:

        N_save = np.loadtxt(path + '/../ParameterFile.par', comments='/*')[8]
        dx = np.loadtxt(path + '/../ParameterFile.par', comments='/*')[7]
        mass = np.loadtxt(path + '/../ParameterFile.par', comments='/*')[4]

        x = np.loadtxt(path + f'/xs.dat')
        NX = len(x)

        f.create_dataset('Xgrid', data=x, chunks=(int(NX/2)), compression="gzip", compression_opts=9)
        f.create_dataset('Mass', data=mass)
        f.create_dataset('dx', data=dx)

        for ii in [0, 1, 2]:
            try:
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore",category=UserWarning)
                    if ii == 0:
                        r = np.loadtxt(path + f'/rho.dat')
                    elif ii == 1:
                        B = np.loadtxt(path + f'/B.dat')
                    elif ii == 2:
                        E = np.loadtxt(path + f'/E.dat')
                    t = np.loadtxt(path + f'/times.dat')
            except:
                continue

            iteration = 0
            i = 0

            for tt, dt, _ in t:
                gr = f.require_group(f'{int(iteration)}')
                if ii == 1:
                    gr.create_dataset('B', data=B[i,:], chunks=(int(NX/2)), compression="gzip", compression_opts=9)
                elif ii == 2:
                    gr.create_dataset('e^b', data=E[i,:], chunks=(int(NX/2)), compression="gzip", compression_opts=9)
                elif ii == 0:
                    gr.create_dataset('rho', data=r[i,:], chunks=(int(NX/2)), compression="gzip", compression_opts=9)
                    gr.create_dataset('t', data=tt)
                    gr.create_dataset('dt', data=dt)

                iteration += N_save
                i += 1

In [16]:
with h5py.File('/home/lorenzo/phd/LQG/FullDynamics/sims/m10.0_dx0.005_xMax70.0_tf400.0_r05.0_a013.3/outputs/output.h5', 'r') as f:
    group_names = list(f.keys())
    rl = []
    for i in range(len(group_names)):
        try:
            int(group_names[i])  # Try converting the element to an integer
        except ValueError:
            rl.append(i)  # Remove the element if it cannot be converted
    for r in rl[::-1]:
        group_names.pop(r)

    group_names = sorted(group_names, key=int)


8001
